# Model to predict SII based on columns: Demographics, Physical Measures, and Sleep Disturbance Scale

In [2]:
# Import necessary packages and libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, normaltest

In [4]:
# Load in data
df_filepath = r"C:\Users\quynh\OneDrive\Documents\DSA 330\Final Project\train (1).csv"
df = pd.read_csv(df_filepath)

In [6]:
print(df.head())

         id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  ...  PCIAT-PCIAT_18  PCIAT-PCIAT_19  PCIAT

In [8]:
print(df.shape)

(3960, 82)


**3960 total rows and 82 total features**

In [17]:
# Group PCIAT questions together for convienence and narrow down data
pciatQ_columns = [f'PCIAT-PCIAT_{i:02d}' for i in range(1, 21)]
pciatQ_df= df[pciatQ_columns]

columns_to_select = [
    'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 
    'Physical-Season', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 
    'Physical-Waist_Circumference', 'Physical-Diastolic_BP', 'Physical-HeartRate', 
    'Physical-Systolic_BP', 'SDS-Season', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 
    'sii', 'PCIAT-PCIAT_Total'
] + list(pciatQ_df.columns)  

# Select the columns from the original DataFrame
qtData = df[columns_to_select]
print(qtData)


     Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0                         Fall                5                0   
1                       Summer                9                0   
2                       Summer               10                1   
3                       Winter                9                0   
4                       Spring               18                1   
...                        ...              ...              ...   
3955                      Fall               13                0   
3956                    Winter               10                0   
3957                      Fall               11                0   
3958                    Spring               13                0   
3959                    Spring               11                0   

     Physical-Season  Physical-BMI  Physical-Height  Physical-Weight  \
0               Fall     16.877316             46.0             50.8   
1               Fall     14.035590     

In [19]:
nan = (qtData.isnull().sum())
print(nan)

Basic_Demos-Enroll_Season          0
Basic_Demos-Age                    0
Basic_Demos-Sex                    0
Physical-Season                  650
Physical-BMI                     938
Physical-Height                  933
Physical-Weight                  884
Physical-Waist_Circumference    3062
Physical-Diastolic_BP           1006
Physical-HeartRate               993
Physical-Systolic_BP            1006
SDS-Season                      1342
SDS-SDS_Total_Raw               1351
SDS-SDS_Total_T                 1354
sii                             1224
PCIAT-PCIAT_Total               1224
PCIAT-PCIAT_01                  1227
PCIAT-PCIAT_02                  1226
PCIAT-PCIAT_03                  1229
PCIAT-PCIAT_04                  1229
PCIAT-PCIAT_05                  1231
PCIAT-PCIAT_06                  1228
PCIAT-PCIAT_07                  1231
PCIAT-PCIAT_08                  1230
PCIAT-PCIAT_09                  1230
PCIAT-PCIAT_10                  1227
PCIAT-PCIAT_11                  1226
P

In [21]:
nanPercent = (nan / 3960) * 100
print(nanPercent)

Basic_Demos-Enroll_Season        0.000000
Basic_Demos-Age                  0.000000
Basic_Demos-Sex                  0.000000
Physical-Season                 16.414141
Physical-BMI                    23.686869
Physical-Height                 23.560606
Physical-Weight                 22.323232
Physical-Waist_Circumference    77.323232
Physical-Diastolic_BP           25.404040
Physical-HeartRate              25.075758
Physical-Systolic_BP            25.404040
SDS-Season                      33.888889
SDS-SDS_Total_Raw               34.116162
SDS-SDS_Total_T                 34.191919
sii                             30.909091
PCIAT-PCIAT_Total               30.909091
PCIAT-PCIAT_01                  30.984848
PCIAT-PCIAT_02                  30.959596
PCIAT-PCIAT_03                  31.035354
PCIAT-PCIAT_04                  31.035354
PCIAT-PCIAT_05                  31.085859
PCIAT-PCIAT_06                  31.010101
PCIAT-PCIAT_07                  31.085859
PCIAT-PCIAT_08                  31

**Physical Waist Circumference** has way too many null values, so we drop this feature

In [24]:
dropNan_df = qtData.drop(columns = ['Physical-Waist_Circumference'])
display(dropNan_df)

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20
0,Fall,5,0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,...,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0
1,Summer,9,0,Fall,14.035590,48.0,46.0,75.0,70.0,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Summer,10,1,Fall,16.648696,56.5,75.6,65.0,94.0,117.0,...,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0
3,Winter,9,0,Summer,18.292347,56.0,81.6,60.0,97.0,117.0,...,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0
4,Spring,18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,Fall,13,0,Fall,16.362460,59.5,82.4,71.0,70.0,104.0,...,2.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0
3956,Winter,10,0,Spring,18.764678,53.5,76.4,60.0,78.0,118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3957,Fall,11,0,Winter,21.441500,60.0,109.8,79.0,99.0,116.0,...,1.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0,0.0,1.0
3958,Spring,13,0,Winter,12.235895,70.7,87.0,59.0,61.0,113.0,...,2.0,0.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0


## Encode Categorical Variables using One Hot Encoding
* Basic_Demos-Enroll_Season
* Basic_Demos-Sex
* Physical-Season
* SDS-Season

In [27]:
encodeData = pd.get_dummies(dropNan_df, columns=['Basic_Demos-Enroll_Season', 'Basic_Demos-Sex', 'Physical-Season', 'SDS-Season'])
display(encodeData)

,Basic_Demos-Age,Physical-BMI,Physical-Height,Physical-Weight,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,SDS-SDS_Total_Raw,SDS-SDS_Total_T,sii,...,Basic_Demos-Sex_0,Basic_Demos-Sex_1,Physical-Season_Fall,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter
0,5,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,2.0,...,True,False,True,False,False,False,False,False,False,False
1,9,14.035590,48.0,46.0,75.0,70.0,122.0,46.0,64.0,0.0,...,True,False,True,False,False,False,True,False,False,False
2,10,16.648696,56.5,75.6,65.0,94.0,117.0,38.0,54.0,0.0,...,False,True,True,False,False,False,True,False,False,False
3,9,18.292347,56.0,81.6,60.0,97.0,117.0,31.0,45.0,1.0,...,True,False,False,False,True,False,False,False,True,False
4,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,13,16.362460,59.5,82.4,71.0,70.0,104.0,35.0,50.0,1.0,...,True,False,True,False,False,False,False,False,False,True
3956,10,18.764678,53.5,76.4,60.0,78.0,118.0,NaN,NaN,NaN,...,True,False,False,True,False,False,False,False,False,False
3957,11,21.441500,60.0,109.8,79.0,99.0,116.0,56.0,77.0,1.0,...,True,False,False,False,False,True,False,False,False,True
3958,13,12.235895,70.7,87.0,59.0,61.0,113.0,33.0,47.0,0.0,...,True,False,False,False,False,True,False,True,False,False


In [29]:
print(encodeData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Basic_Demos-Age                   3960 non-null   int64  
 1   Physical-BMI                      3022 non-null   float64
 2   Physical-Height                   3027 non-null   float64
 3   Physical-Weight                   3076 non-null   float64
 4   Physical-Diastolic_BP             2954 non-null   float64
 5   Physical-HeartRate                2967 non-null   float64
 6   Physical-Systolic_BP              2954 non-null   float64
 7   SDS-SDS_Total_Raw                 2609 non-null   float64
 8   SDS-SDS_Total_T                   2606 non-null   float64
 9   sii                               2736 non-null   float64
 10  PCIAT-PCIAT_Total                 2736 non-null   float64
 11  PCIAT-PCIAT_01                    2733 non-null   float64
 12  PCIAT-

## Impute Remaining Nan Values

**Categorical**: Replace with Mode
* Basic_Demos-Sex_0
* Basic_Demos-Sex_1
* Physical-Season_Fall
* Physical-Season_Spring
* Physical-Season_Summer
* Physical-Season_Winter
* SDS-Season_Fall
* SDS-Season_Spring
* SDS-Season_Summer
* SDS-Season_Winter
* PCIAT-PCIAT_01 - PCIAT-PCIAT_20

**Numerical**: Replace with Mean or Median depending on distrubutions
* Basic_Demos-Age
* Physical-BMI
* Physical-Height
* Physical-Weight
* Physical-Diastolic_BP
* Physical-HeartRate
* Physical-Systolic_BP
* PCIAT-PCIAT_Total
* SDS-SDS_Total_Raw
* SDS-SDS_Total_T
* sii

**Before imputing, we have to check the distributions**

* Run Normality Tests -> Shapiro-Wilk and D-Agostino-Pearson Normality Tests

**Hypothesis**
* H0: Data is normal
* Ha: Data is not normal

In [88]:
# Perform Shapiro-Wilk and D'Agostino-Pearson tests
numerical_features = ['Basic_Demos-Age', 'Physical-BMI', 'Physical-HeartRate', 
                      'Physical-Height', 'Physical-Weight', 'Physical-Diastolic_BP','Physical-Systolic_BP',
                      'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
                      'PCIAT-PCIAT_Total','sii']

results = []
for column in numerical_features:
    shapiro_stat, shapiro_p = shapiro(encodeData[column].dropna())  # Shapiro-Wilk
    dagostino_stat, dagostino_p = normaltest(encodeData[column].dropna())  # D’Agostino
    
    results.append({
        'Feature': column,
        'Shapiro_Statistic': shapiro_stat,
        'Shapiro_P_Value': shapiro_p,
        'Dagostino_Statistic': dagostino_stat,
        'Dagostino_P_Value': dagostino_p,
        'Normal (Shapiro)': shapiro_p >= 0.05,
        'Normal (Dagostino)': dagostino_p >= 0.05
    })

normality_results = pd.DataFrame(results)

print(normality_results)

                  Feature  Shapiro_Statistic  Shapiro_P_Value  \
0         Basic_Demos-Age           0.944018     2.361625e-36   
1            Physical-BMI           0.864608     2.696282e-45   
2      Physical-HeartRate           0.994498     4.174614e-09   
3         Physical-Height           0.982142     4.343394e-19   
4         Physical-Weight           0.915481     2.026639e-38   
5   Physical-Diastolic_BP           0.926499     8.361488e-36   
6    Physical-Systolic_BP           0.940492     6.587958e-33   
7       SDS-SDS_Total_Raw           0.922664     1.152260e-34   
8         SDS-SDS_Total_T           0.929645     2.311065e-33   
9       PCIAT-PCIAT_Total           0.956972     1.028554e-27   
10                    sii           0.724639     2.577268e-55   

    Dagostino_Statistic  Dagostino_P_Value  Normal (Shapiro)  \
0            279.386517       2.147775e-61             False   
1           1055.468114      6.427042e-230             False   
2             43.796792    

**Since the P-values for all the features are less than alpha, we rejet the null hypothesis. None of the features are normally distributed, so we'll go ahead and impute with median for all numerical features and sii**

We make a copy of the dataframe to not impact the original, then impute 

In [94]:
replaceMode = [
    'Basic_Demos-Sex_0', 'Basic_Demos-Sex_1',
    'Physical-Season_Fall', 'Physical-Season_Spring', 
    'Physical-Season_Summer', 'Physical-Season_Winter',
    'SDS-Season_Fall', 'SDS-Season_Spring', 
    'SDS-Season_Summer', 'SDS-Season_Winter',
    'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02','PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05' ,
    'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',  'PCIAT-PCIAT_09' ,'PCIAT-PCIAT_10' ,
    'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12' ,'PCIAT-PCIAT_13',  'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15',
    'PCIAT-PCIAT_16' , 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19' , 'PCIAT-PCIAT_20'  
]

replaceMedian = [
    'Basic_Demos-Age', 'Physical-BMI', 'Physical-Height', 
    'Physical-Weight', 'Physical-Diastolic_BP', 
    'Physical-HeartRate', 'Physical-Systolic_BP',  'PCIAT-PCIAT_Total' ,
    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'sii'
]

finaldf = encodeData.copy()

# Imputing starts
for column in replaceMode:
    if column in finaldf.columns:  # Check if column exists in the DataFrame
        finaldf[column] = finaldf[column].fillna(finaldf[column].mode()[0])

# Impute median for numerical columns
for column in replaceMedian:
    if column in finaldf.columns:  # Check if column exists in the DataFrame
        finaldf[column] = finaldf[column].fillna(finaldf[column].median())

# Print the resulting DataFrame
print("Data after imputation:")
print(finaldf)


Data after imputation:
      Basic_Demos-Age  Physical-BMI  Physical-Height  Physical-Weight  \
0                   5     16.877316             46.0             50.8   
1                   9     14.035590             48.0             46.0   
2                  10     16.648696             56.5             75.6   
3                   9     18.292347             56.0             81.6   
4                  18     17.937682             55.0             77.0   
...               ...           ...              ...              ...   
3955               13     16.362460             59.5             82.4   
3956               10     18.764678             53.5             76.4   
3957               11     21.441500             60.0            109.8   
3958               13     12.235895             70.7             87.0   
3959               11     17.937682             55.0             77.0   

      Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
0                      68.0

In [96]:
print(finaldf.isnull().sum())

Basic_Demos-Age                     0
Physical-BMI                        0
Physical-Height                     0
Physical-Weight                     0
Physical-Diastolic_BP               0
Physical-HeartRate                  0
Physical-Systolic_BP                0
SDS-SDS_Total_Raw                   0
SDS-SDS_Total_T                     0
sii                                 0
PCIAT-PCIAT_Total                   0
PCIAT-PCIAT_01                      0
PCIAT-PCIAT_02                      0
PCIAT-PCIAT_03                      0
PCIAT-PCIAT_04                      0
PCIAT-PCIAT_05                      0
PCIAT-PCIAT_06                      0
PCIAT-PCIAT_07                      0
PCIAT-PCIAT_08                      0
PCIAT-PCIAT_09                      0
PCIAT-PCIAT_10                      0
PCIAT-PCIAT_11                      0
PCIAT-PCIAT_12                      0
PCIAT-PCIAT_13                      0
PCIAT-PCIAT_14                      0
PCIAT-PCIAT_15                      0
PCIAT-PCIAT_

## Random Forest Classifier

**Split features and target**

In [105]:
x = finaldf.drop(columns=['sii'])
y = finaldf['sii']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9974747474747475

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       560
         1.0       0.99      1.00      1.00       148
         2.0       0.99      0.99      0.99        76
         3.0       1.00      0.88      0.93         8

    accuracy                           1.00       792
   macro avg       1.00      0.97      0.98       792
weighted avg       1.00      1.00      1.00       792



In [107]:
from sklearn.metrics import cohen_kappa_score

def sii(y):
    """
    0-30=None; 31-49=Mild; 50-79=Moderate; 80-100=Severe
    """
    # y = y[y_cols]
    return np.digitize(y.sum(axis=1), bins=[30, 50, 80], right=True)

def compare_sii(y1, y2):
    return cohen_kappa_score(sii(y1), sii(y2), weights='quadratic')

In [109]:
cohen_kappa_score(y_test, y_pred, weights='quadratic')

0.9974244907515805

In [111]:
y_test

149     0.0
1025    0.0
1846    0.0
720     0.0
325     0.0
       ... 
1226    0.0
736     0.0
3292    1.0
927     0.0
3778    0.0
Name: sii, Length: 792, dtype: float64